# Previsão da Progressão dos Casos Confirmados de COVID19 por Estado no Brasil

### 

In [1]:
#Ocult Warnings
import warnings
warnings.filterwarnings("ignore")

#Memory
import memory_profiler

In [2]:
%%time

#Data Wrangling
from datetime import datetime as dt
import pandas as pd
import numpy as np

#Data Transform
from category_encoders import OneHotEncoder

#Web Scraping
from tabulate import tabulate
import urllib.request
from bs4 import BeautifulSoup
import requests

#Machine Learning - Algorithms
from sklearn.linear_model import Ridge

#Machine Learning - Save
from sklearn.externals import joblib

#Carregando Dados
df = pd.read_csv("C:/Users/KÁTIA/Desktop/EVOLUTION 2020/COVID/UPDATE/brazil_covid19.csv",sep=',')

#Removendo coluna 'region'
df=df.drop(columns='region')

#Convertendo 'date' de string para datetime
df.date=pd.to_datetime(df.date)

#Ordenando dartagrame por estado e data
df=df.sort_values(by=['state','date'])

#Retirando dados da web
res=requests.get('https://www.estadosecapitaisdobrasil.com/')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]

#Criando dataframe
uf_reg = pd.read_html(str(table))
uf_reg=pd.DataFrame(uf_reg)
uf_reg=uf_reg[0][0]
uf_reg.head(2)

#Renomeando e definindo com índice a coluna chave do dataframe
uf_reg=uf_reg.rename(columns={'Estado':'state','Sigla':'UF'})
uf_reg=uf_reg.set_index(uf_reg.state)

#Juntando os datasets
df=df.merge(uf_reg[['Região','UF']],how="left",on='state')

#Datas únicas de forma ordenada
df.date.unique()

#Criando dataframe
start=pd.DataFrame(np.arange(1,len(df.date.unique())+1),index=df.date.unique(),columns=['Day'])
start.index.name='date'

#Juntando os dataframes
df=df.merge(start['Day'], how='left', on='date')

#Copiando dataframe original
date_state=df.copy()

#Relacionadas ao dia anterior (d-1)
#Número de Casos Confirmados por Estado
date_state['Cases_Last_Day']=date_state.groupby(['state']).cases.shift()

#Criando novas colunas para transformação
date_state['UF_CAT']=date_state.UF

#Transformando as variáveis
ohe_uf = OneHotEncoder(cols=['UF_CAT'],use_cat_names=True,drop_invariant=True)
date_state = ohe_uf.fit_transform(date_state)

#Substituindo dados nulos por 0
date_state=date_state.fillna(0)

#Último dia
date_state.Day.max()

#Divisão dos dados
#Dias de início e fim, respecivamente
mn, mx = date_state.Day.max()-31 , date_state.Day.max()

#Root Mean Squared Logarithmic Error
def RMSLE(pred,target):
    
    pred=np.array(pred)
    target=np.array(target)
    
    return np.sqrt(np.mean((np.log1p(abs(pred))-np.log1p(abs(target)))**2))

#Selecionando as features necessárias
features = ['Cases_Last_Day', 'UF_CAT_TO', 'UF_CAT_SE', 'UF_CAT_AL', 'UF_CAT_MS', 'UF_CAT_GO', 'UF_CAT_AC']

models=[Ridge(random_state=0,alpha=0.01,normalize=True,fit_intercept=True)]

name_mod=['RID']

feat_mod = [features]

for num in range(0,len(models)):
    
    error2=[]
    
    for i in range(mn,mx+1):
        if date_state[date_state['Day']==i].shape[0] !=0:

            train=date_state[date_state['Day']<i]
            test=date_state[date_state['Day']==i]

            sel=feat_mod[num]
            X_train=train.loc[:,sel].values
            y_train=train['cases'].values
            X_test=test.loc[:,sel].values
            y_test=test['cases'].values

            model=models[num]
            model.fit(X_train,y_train)

            y_pred=model.predict(X_test)
                
            e2 = RMSLE(y_pred,y_test)
            error2.append(e2)              

    #Imprimir mensagem para cada algorimo mostrando os scores
    #print("\n{}\nMédia do Erro RMSLE: {:.4f}\n".format(name_mod[num],np.mean(error2)))
    
    #Salvando o modelo
    joblib.dump(model,'model_{}.pkl'.format(name_mod[num]))

#Carregando e visualizando os parâmetros do modelo
model=joblib.load('model_RID.pkl')
#print(model,'\n\n',model.coef_,'\n')
print('Mémoria\n{} Mb'.format(memory_profiler.memory_usage(-1)[0]),'\n\nTempo')

Mémoria
129.07421875 Mb 

Tempo
Wall time: 6.49 s
